<b>Вспомогательный проект по курсу "Машинное обучение для текстов"</b><br>
Задача классификации тональности текста<br>
подключим библиотеки

In [67]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import base64
from IPython.display import HTML

Загрузим файлы датасета

In [41]:
df_train = pd.read_csv('~/Documents/datasets/tweets_lemm_train.csv')
df_test = pd.read_csv('~/Documents/datasets/tweets_lemm_test.csv')

Посмотрим на данные

In [42]:
df_train.head()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


In [43]:
df_test.head()

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [44]:
print(df_train.shape)
print(df_test.shape)

(5000, 3)
(3000, 2)


In [45]:
X_train = df_train['lemm_text'].values.astype('U')
X_test = df_test['lemm_text'].values.astype('U')

Обучающая выборка

In [46]:
print(X_train)

['хоть я и школотый но поверь у мы то же самый общество профилировать предмет тип '
 'да весь таки он немного похожий на он но мой мальчик весь равно хороший '
 'ну ты идиотка я испугаться за ты ' ... 'черта ну зачем они переделать '
 'вроде бы и любим друг друг и быть вместе не мочь не дать бог вы такой '
 'один из самый скучный суббота за последний время так и не вырваться на улица за свежий воздух ']


Тестовая выборка

In [47]:
print(X_test)

['если крис так интересовать ребёнок то либо они он просто умилять либо он педофил '
 'по рубль в месяц можно разместить ссылка на тележка в гиперспар в пролетарский торопиться предложение ограничить '
 'учитывать что сейчас преобладать один половина ник то сопротивление бесполезный как я понимать '
 ... 'на улица пусто ни человек ни машина наверно холод бояться '
 'по мой мы и ходить за водка в три час за это что бы обработать быть чем правда выпить она весь '
 'р в сказать что с утро быть самостоятельный ну капец что с он происходить злой постоянно ']


"Соберём" целевой признак

In [48]:
y_train = df_train['positive']
y_train.head()
print(y_train.shape)

(5000,)


Подключим стоп слова

In [49]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavelpankov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создадим TF-IDF для корпуса обучающей выборки

In [50]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
X_train_idf = count_tf_idf.fit_transform(X_train)
X_train_idf.shape

(5000, 9737)

Создадим и обучим линейную регрессию

In [60]:
model = LogisticRegression()
model.fit(X_train_idf, y_train)

LogisticRegression()

Создали и обучили модель<br>
Проверим модель на обучающих данных

In [62]:
predictions = model.predict(X_train_idf)
accuracy = accuracy_score(y_train, predictions)
print('Accuracy на обучающей выборке:', accuracy)

Accuracy на обучающей выборке: 0.8984


Модель получилась сравнительно точной, ошибается в примерно в 10% случаев. Для анализа токсичности твитов, конечно, хотелось бы сильно точнее, но задание просит не ниже 0.62, так что погнали дальше<br>
Запустим модель на тестовых данных, добавим предсказания в датафрейм

In [63]:
X_test_idf = count_tf_idf.transform(X_test)
predictions = model.predict(X_test_idf)
df_test['positive'] = predictions

Посмотрим, сколько в тестовых данных модель определила позитивных и негативных твитов

In [66]:
print('Негативных твитов:', df_test.query('positive == 0').shape)
print('Позитивных твитов:', df_test.query('positive == 1').shape)

Негативных твитов: (1436, 3)
Позитивных твитов: (1564, 3)


Почти поровну. Сохраним файл, чтобы его потом отправить на проверку

In [68]:
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [69]:
create_download_link(df=df_test, filename='predictions')